In [19]:
import cv2
import numpy as np

In [30]:
cap = cv2.VideoCapture('test_short.mp4')

while(cap.isOpened()):
    ret, frame = cap.read()
    
    if frame is None:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    cv2.imshow('frame',gray)
    # Framerate is 29.1 FPS -> 1000 / 29.1 = 34
    if cv2.waitKey(34) & 0xFF == ord('q'):
        break

print("done")
cv2.destroyWindow('frame')
cv2.waitKey(1)
cap.release()

done


error: OpenCV(4.2.0) /Users/travis/build/skvark/opencv-python/opencv/modules/highgui/src/window_QT.cpp:539: error: (-27:Null pointer) NULL guiReceiver (please create a window) in function 'cvDestroyWindow'


In [25]:
# framerate is 29.1 FPS
# Now, we will cut the video to only 5 seconds for easier use of this stuff.
cap = cv2.VideoCapture('test_video.mp4')

# Define codec, create writer to write the video
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('test_short.mp4', fourcc, 29, (240, 352))

ct_max = 29 * 5
ct = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    ct += 1
    if ct >= ct_max:
        break
    
    if ret==True:

        # write the flipped frame
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [21]:
###      BELOW COPIED FROM TUTORIAL      ###

cap = cv2.VideoCapture('test_video.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()